# Klasy

In [142]:
class CountedObject(object):
    count = 0   # statyczna składowa
    
    def __init__(self):
        CountedObject.count += 1
    
    @staticmethod  # statyczna metoda
    def get_count():
        return CountedObject.count

In [143]:
lst = [CountedObject() for i in range(10)]

In [144]:
CountedObject.count

10

In [145]:
CountedObject.get_count()

10

In [ ]:
lst[3].count = 42

In [150]:
class Person:
    name = "unknown"

In [151]:
p1 = Person()

In [152]:
p1.name

'unknown'

In [153]:
p1.name = "Jan"

In [154]:
p1.name

'Jan'

In [155]:
class Date:
    year = 2023

    def __init__(self, day, month, year = None):
        self.day = day
        self.month = month
        if year:
            self.year = year 
    
    @classmethod
    def from_string(cls, date_as_string):        
        day, month, year = date_as_string.split('-')
        return cls(int(day), int(month), int(year)) # utworzenie instancji klasy cls

    @classmethod
    def update_default_year(cls, value):
        cls.year = value

In [157]:
Date.year

2023

In [158]:
d1 = Date(25, 9)

In [159]:
d1.year

2023

In [160]:
d1.__dict__

{'day': 25, 'month': 9}

In [161]:
Date.update_default_year(2024)

In [162]:
d1.year

2024

In [165]:
d2 = Date.from_string("25-9-2023")

In [166]:
d2.__dict__

{'day': 25, 'month': 9, 'year': 2023}

In [167]:
d1.from_string("3-3-2022")

# Deskryptor

## Non-data descriptor

In [ ]:
import os

class DirectorySize:
    def __get__(self, instance, owner_class):
        print(f'Access to {instance} using descriptor {self}')
        return len(os.listdir(instance.directory_name))
    

class Directory:
    size = DirectorySize() # descriptor instance

    def __init__(self, directory_name):
        self.directory_name = directory_name # regular instance attribute

In [ ]:
local_dir = Directory('.')

In [ ]:
local_dir.__dict__

{'directory_name': '.'}

In [ ]:
local_dir.size

Access to <__main__.Directory object at 0x7f08f957dc10> using descriptor <__main__.DirectorySize object at 0x7f08f9535250>


1

## Data descriptor

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        logging.info('Setting names: %r and %r', self.public_name, self.private_name)

    def __get__(self, instance, owner_class=None):
        value = getattr(instance, self.private_name)
        logging.info('Accessing %r.%r giving %r', instance, self.public_name, value)
        return value

    def __set__(self, instance, value):
        logging.info('Updating %r.%r to %r', instance, self.public_name, value)
        setattr(instance, self.private_name, value)


class Person:
    age = LoggedAccess()             # Descriptor instance
    name = LoggedAccess()

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()

INFO:root:Setting names: 'age' and '_age'
INFO:root:Setting names: 'name' and '_name'


In [ ]:
p1 = Person("Jan", 22)

INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'name' to 'Jan'
INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'age' to 22


In [ ]:
p1.age

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'age' giving 22


22

In [ ]:
p1.age = 44

INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'age' to 44


In [ ]:
p1.age

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'age' giving 44


44

In [ ]:
p1.name

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'name' giving 'Jan'


'Jan'

In [ ]:
p1.birthday()

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'age' giving 44
INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'age' to 45


### ReadOnlyProperty

In [201]:
class ReadOnlyProperty:
    def __init__(self, fget):
        self.fget = fget

    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("Attribute is read-only")

In [214]:
class Data:
    def __init__(self, data: int, name: str = "default") -> None:
        self._data = data
        self._name = name

    # def data(self) -> int:
    #    return self._data
    
    # data = ReadOnlyProperty(data)

    @ReadOnlyProperty
    def data(self):
        return self._data
    
    @property
    def name(self):
        return self._name

In [215]:
data1 = Data(42)

In [216]:
data1.data

42

In [217]:
data1.data = 665

AttributeError: Attribute is read-only